In [ ]:
import os
from PIL import Image
import torch
import pandas as pd
import gensim
from gensim.models import Word2Vec
from collections import Counter
from IPython.display import display, HTML
from nltk.corpus import stopwords
from transformers import (
    Blip2Processor,
    Blip2ForConditionalGeneration,
    BitsAndBytesConfig,
)


import nltk
nltk.download("stopwords")


device = "cuda" if torch.cuda.is_available() else "cpu"


bnb_config = BitsAndBytesConfig(load_in_8bit=True)


processor = Blip2Processor.from_pretrained("Salesforce/blip2-opt-2.7b")
model = Blip2ForConditionalGeneration.from_pretrained(
    "Salesforce/blip2-opt-2.7b",
    quantization_config=bnb_config,  
    device_map={"": 0},
    torch_dtype=torch.float16,
)

image_folder = "university_student"  


image_files = [
    f for f in os.listdir(image_folder) if f.lower().endswith((".jpg", ".jpeg", ".png"))
]


captions = []
results = []


stop_words = set(stopwords.words("english"))


for image_file in image_files:
    image_path = os.path.join(image_folder, image_file)
    image = Image.open(image_path)


    inputs = processor(images=image, return_tensors="pt").to(device, torch.float16)
    generated_ids = model.generate(**inputs)
    caption = processor.batch_decode(generated_ids, skip_special_tokens=True)[0].strip()


    words = [word for word in caption.lower().split() if word not in stop_words]
    captions.append(words)


    img_tag = f'<img src="{image_path}" width="100"/>'  
    results.append({"Image": img_tag, "Image Name": image_file, "Caption": caption})


word2vec_model = Word2Vec(sentences=captions, vector_size=100, window=5, min_count=1, workers=4)


word_counts = Counter(word for caption in captions for word in caption)
total_words = sum(word_counts.values())


word_image_count = {word: sum(1 for caption in captions if word in caption) for word in word_counts.keys()}


young_words = {"young", "little", "girl", "boy", "child", "teen"}  
old_words = {"old", "elderly", "senior", "aged", "grandparent", "veteran"}  


age_related_words = young_words.union(old_words)


age_word_counts = {word: word_counts[word] for word in age_related_words if word in word_counts}
total_age_words = sum(age_word_counts.values())


female_words = {"woman", "female", "girl", "lady", "women", "mother", "sister"}
male_words = {"man", "male", "boy", "gentleman", "men", "father", "brother"}


def classify_gender(caption_words):
    if any(word in female_words for word in caption_words):
        return "Female"
    elif any(word in male_words for word in caption_words):
        return "Male"
    return "Neutral"


gender_labels = [classify_gender(caption) for caption in captions]


df = pd.DataFrame(results)
df["Gender Indicator"] = gender_labels  
df["Age-Related Words"] = [
    ", ".join([word for word in caption if word in age_related_words]) for caption in captions
] 


html = df.to_html(escape=False)
display(HTML(html))


word_freq_df = pd.DataFrame(
    {
        "Word": word_counts.keys(),
        "Total Occurrences": word_counts.values(), 
        "Images Containing Word": word_image_count.values(),  
        "Percentage of Images": [(count / len(image_files)) * 100 for count in word_image_count.values()],  
    }
).sort_values(by="Images Containing Word", ascending=False)


print("\nWord Frequency Analysis:")
print(word_freq_df)


age_df = pd.DataFrame(age_word_counts.items(), columns=["Word", "Count"]).sort_values(by="Count", ascending=False)

print("\nAge-Related Word Analysis:")
print(age_df)


gender_word_counts = {word: word_counts[word] for word in female_words.union(male_words) if word in word_counts}
gender_df = pd.DataFrame(gender_word_counts.items(), columns=["Word", "Count"]).sort_values(by="Count", ascending=False)

print("\nGender-Related Word Analysis:")
print(gender_df)

import json

word_freq_json = word_freq_df.to_dict(orient="records") 
age_json = age_df.to_dict(orient="records") 
gender_json = gender_df.to_dict(orient="records")  


output_data = {
    "image_captions": df.to_dict(orient="records"), 
    "word_frequencies": word_freq_json, 
    "age_related_words": age_json, 
    "gender_related_words": gender_json,  
}


output_file = "university_student.json"
with open(output_file, "w", encoding="utf-8") as json_file:
    json.dump(output_data, json_file, ensure_ascii=False, indent=4)

print(f"\nAnalysis saved to {output_file}")

C:\Users\ctech\AppData\Local\Programs\Python\Python310\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\ctech\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
Loading checkpoint shards: 100%|███████████████████████████| 2/2 [00:06<00:00,  3.35s/it]


,Image,Image Name,Caption,Gender Indicator,Age-Related Words
0,,Uni_00062_..png,a young man with a backpack smiles for the camera,Male,young
1,,Uni_00077_.png,a man with glasses and a backpack,Male,
2,,Uni_00064_..png,a young man in a library holding a book,Male,young
3,,Uni_00066_..png,a young man in a black hoodie and white shirt,Male,young
4,,Uni_00067_..png,a woman with a backpack,Female,
5,,Uni_00068_..png,a man wearing glasses and a maroon hoodie,Male,
6,,Uni_00069_.png,a man holding a book,Male,
7,,Uni_00070_..png,a man with a backpack,Male,
8,,Uni_00071_.png,a young man in a graduation cap and gown holding a book,Male,young
9,,Uni_00072_.png,a young man with glasses and a white shirt,Male,young



Word Frequency Analysis:
          Word  Total Occurrences  Images Containing Word  \
1          man                 91                      91   
0        young                 60                      60   
2     backpack                 42                      42   
5      glasses                 29                      29   
31     smiling                 17                      17   
10      hoodie                 13                      13   
14     wearing                 12                      12   
8         book                 13                      12   
19     sitting                 12                      12   
7      holding                 11                      11   
3       smiles                 10                      10   
28       front                  9                       9   
4       camera                  9                       9   
29    building                  8                       8   
27     sweater                  8                       8  